In [499]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import random

# Projet: Modèles linéaires:  Adaline et Regression Logistique

Nous allons nous intéresser à l'implémentation d'un algorithme de descente de gradient pour trouver le meilleur paramètre d'un module Adaline ou de regression logistique.

Pout cela, on implémentera un algorithme de descente de gradient stochastique que nous avons vu au TP précédent et dont le pseudo-code peut être résumé comme suit:

```input: Train, eta, m, MaxEp, modele
init : w
epoque=0
while epoque<=MaxEp
    choisir un exemple (x,y) de Train de façon aléatoire
    calculer h = w*x
    calculer Loss(h, y)
    w <- w - eta*"gradient de Loss(h, y) par rapport à w"
    epoque <- epoque+1
output: w
```
où "eta" est le pas de la descente de gradient (exemple: eta=0.01).

Si on veut imprimer l'erreur tous les "m" pas de gradient:
```input: Train, eta, m, MaxEp, modele
init : w
epoque=0
while epoque<=MaxEp
    err = 0
    for i in range(m):
        choisir un exemple (x,y) de Train de façon aléatoire
        calculer h = w*x
        err += Loss(h, y)
        w <- w - eta*"gradient de Loss(h, y) par rapport à w"
    epoque <- epoque+1
    print(err)
output: w
```

Pour un poids $w$, on définit $h_\mathbf{w}(\mathbf{x})=w_0x_0+w_1x_1+...w_dx_d$. Pour chacun des deux modèles, et pour un exemple $(\mathbf{x},y)$, la prédiction $\hat{y}(\mathbf{w}, \mathbf{x})$ et la fonction de coût  $\mathcal{L}(\mathbf{w}, \mathbf{x})$ sont: 
- Adaline: $\hat{y}(\mathbf{w}, \mathbf{x}) = h_\mathbf{w}(x)$ et $$\mathcal{L}(\mathbf{w})=(y-\hat{y}(\mathbf{w},\mathbf{x}))^2=(y-h_\mathbf{w}(\mathbf{x}))^2,$$
- Régression logistique: $\hat{y}(w, x) = 1/(1+e^{-h_{\mathbf{w}}(\mathbf{x})})$ et $$\mathcal{L}(\mathbf{w}, x) = - y \log \hat{y}(\mathbf{w},\mathbf{x}) - (1-y)\log(1-\hat{y}(\mathbf{w},\mathbf{x})) = \log(1+e^{h_{\mathbf{w}}(\mathbf{x})})-yh_\mathbf{w}(\mathbf{x}),$$



In [500]:
###############################################
# Codage des fontions de perte et de gradient #
###############################################

# Adaline #
def h(w,x): # Prediction
    """
    w : vecteur de poids
    x : vecteur d'entrée
    Retourne le produit scalaire de w et x
    """
    return w@x

def loss_adaline(p,y):
    """
    p : prédiction
    y : valeur réelle
    Retourne la perte quadratique entre p et y
    """
    return (y-p)**2

def adaline_grad(w,x,y):
    """
    w : vecteur de poids
    x : vecteur d'entrée
    y : valeur réelle
    Retourne le gradient de la perte quadratique entre h(w,x) et y
    """
    return 2*(h(w,x) - y)*x



# Regression logistique #
def prediction_regression_logistique(w,x):
    """
    w : vecteur de poids
    x : vecteur d'entrée
    Retourne la prédiction de la régression logistique
    """
    return 1/(1 + np.exp(-h(w,x)))

def loss_regression_logistique(p,y):
    """
    p : prédiction
    y : valeur réelle
    Retourne la perte de la régression logistique
    """
    p = np.clip(p, 1e-10, 1-1e-10) # Pour éviter les erreurs de calcul
    return -y*np.log(p) - (1-y)*np.log(1-p)

def regression_logistique_grad(w,x,y):
    """
    w : vecteur de poids
    x : vecteur d'entrée
    y : valeur réelle
    Retourne le gradient de la perte de la régression logistique
    """
    return (prediction_regression_logistique(w,x)-y)*x

# Gradient
def gradient(grad, w, x, y):
    """
    Retourne le gradient de la fonction de perte
    """
    return grad(w,x,y)

Nous avons vu les gradients de ces fonctions en TD.

## Partie 1: implémentation de l'algorithme et exemple du "ET logique"

<font color='red'><b>Question 1:</b> le "ET logique".</font> Créer une liste de 4 éléments où chaque élément est un couple de la forme `[x,y]`, avec `x=[1,x1,x2]` et `y = x1 and x2`. Il y a 4 éléments car `x1` et `x2` peuvent chacun prendre la valeur `0` ou `1` (chacun de ces 4 éléments est une liste dont le premier élément est les attributs de l'exemple et le deuxième élément est la classe de l'exemple).

In [501]:
################################
# Creation de liste ET logique #
################################

liste = [0] * 4
for x1 in range(2):
    for x2 in range(2):
        liste[x1*1+x2*2] = [[1,x1,x2], x1 and x2] 

x_ET_logique = np.array([ei[0] for ei in liste])
y_ET_logique = np.array([ei[1] for ei in liste])

x_ET_logique, y_ET_logique


(array([[1, 0, 0],
        [1, 1, 0],
        [1, 0, 1],
        [1, 1, 1]]),
 array([0, 0, 0, 1]))

<font color='red'><b>Question 2:</b></font> Coder un algorithme de descente de gradient stochastique pour les modèles Adaline et le modèle de régression logistique et le faire tourner sur le modèle de "ET logique". 


In [502]:
#############################
# Fonction Gradient Descent #
#############################

import random as rd
    
# Stochastic Gradient Descent #
def SGD(X, Y, Maxep, eta, p, loss_function, g, err_printing=True):
    """
    Descente de gradient stochastique
    
    Entrees:
    X : tableau des features
    Y : tableau des labels / target variable
    Maxep : nombre d'epoques
    eta : fonction de taux d'apprentissage (learning rate)
    p : fonction de prediction
    loss_function : fonction de perte
    g : fonction de gradient
    err_printing : Boolean affichage des erreurs (True par defaut)
    
    Sorties:
    w : vecteur des poids
    """
    w = np.random.rand(len(X[0])) * 0.01
    for epoque in range(Maxep):
        i = rd.randint(0, len(Y) - 1)
        pred = p(w, X[i])
        if err_printing:
            print(loss_function(pred, Y[i]))
        w = w - eta(epoque) * g(w, X[i], Y[i])
    return w

def SGD_with_error_printing(X, Y, Maxep, eta, m, p, loss_function, g, err_printing=True):
    """
    Pareil que SGD mais la mise à jour des poids est faite m fois par époque
    L'erreur correspond à la somme sur les m exemples traités durant l'époque
    """
    w = np.random.rand(len(X[0])) # * 0.01
    for epoque in range(Maxep):
        err = 0
        for _ in range(m):
            i = rd.randint(0, len(Y) - 1)
            pred = p(w, X[i])
            err += loss_function(pred, Y[i])
            w = w - eta(epoque) * g(w, X[i], Y[i])
        if err_printing:
            print("epoque ", epoque, ": ", err)
    return w

# Learning rate #   
def eta_001(t):
    return 0.01

def eta_01(t):
    return 0.1
#################


# Training Adaline on "ET logique"
w_adaline = SGD(x_ET_logique, y_ET_logique, 100, eta_001, h, loss_adaline, adaline_grad, False)
print("Final weights for Adaline on 'ET logique':", w_adaline)

# Training Logistic Regression on "ET logique"
w_logistic = SGD_with_error_printing(x_ET_logique, y_ET_logique, 100, eta_01, 2, prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, False)
print("Final weights for Logistic Regression on 'ET logique':", w_logistic)


Final weights for Adaline on 'ET logique': [0.03685315 0.21740134 0.20505566]
Final weights for Logistic Regression on 'ET logique': [-1.84737059  1.00289146  1.00848033]


Calculer le taux d'erreur de votre algorithme sur cette base (où une erreur est comptabilisé si la prédiction est plus proche de la fausse classe que de la vraie classe). 

In [503]:
#############################
# Calculer le taux d'erreur #
#############################

def calculate_error_rate(w, X, Y, prediction_function, rate_printing=False):
    """
    Calculer le taux d'erreur
    
    Entrees:
    w : vecteur des poids
    X : tableau des features
    Y : tableau des labels / target variable
    prediction_function : fonction de prediction
    rate_printing : Boolean affichage le taux d'erreur (False par defaut)
    
    Sorties:
    errors : taux d'erreur
    """
    errors = 0
    for i in range(len(Y)):
        pred = prediction_function(w, X[i])
        if rate_printing:
            print(f"P: {pred}, Y: {Y[i]}")
        if (pred >= 0.5 and Y[i] == 0) or (pred < 0.5 and Y[i] == 1):
            errors += 1
    return errors / len(Y)


# Function to calculate error rate multiple times
def calculate_multiple_error_rates(X, Y, num_iterations, model, prediction_function, loss, grad, eta, Maxep):
    """
    Calculer le taux d'erreur plusieurs fois
    
    Objectif:
    - Entrainer le modèle plusieurs fois
    - Calculer le taux d'erreur à chaque itération
    - Verifier la stabilité et la convergence de l'algorithme
    """
    if model == 'adaline':
        print("Training Adaline...") 
    elif model == 'logistic':
        print("Training Logistic Regression...")

    for _ in range(num_iterations):
        if model == 'adaline':
            w = SGD(X, Y, Maxep, eta, prediction_function, loss, grad, False)
        elif model == 'logistic':
            w = SGD(X, Y, Maxep, eta, prediction_function, loss, grad, False)
        error_rate = calculate_error_rate(w, X, Y, prediction_function, False)
        print("Error rate:", error_rate)


# Calculate error rates for Adaline and Logistic Regression
num_iterations = 25

#####################################
# Calculate error rates for Adaline #
# calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'adaline', h, loss_adaline, adaline_grad, eta_001, 100)
# calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'adaline', h, loss_adaline, adaline_grad, eta_01, 100)

#################################################
# Calculate error rates for Logistic Regression #
# calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'logistic', prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, eta_001, 100)
# calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'logistic', prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, eta_01, 100)
# calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'logistic', prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, eta_01, 200)
# calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'logistic', prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, eta_01, 300)

## Partie 2: premiers tests avec une base de donnée réelle

<font color='red'><b>Question 3:</b></font> Nous allons maintenant nous intéresser au comportement de ces modèles sur la base SONAR de la collection UCI (http://archive.ics.uci.edu/ml/index.php). Cette base contient 208 exemples en dimension 60 séparés par `,` et la dernière élément correspond à la classe de l'exemple.

    1. Télécharger la collection avec la fonction read_table de la librairie pandas (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html). Les options nécessaires sont `sep=','` et `header=None`  
    2. Créer une liste de listes correspondant à la collection; pour cela initialiser la première liste et en parcourant chaque ligne de la matrice de données; créer une liste associée en remplaçant le dernier élément par `0` ou `+1` et insérer la dans la première liste. 
    Indication: Utiliser la fonction `loc`. 
    3. Écrire une fonction qui génère deux listes de données `x_train` (75%) and `x_test` (25%) en la mélangeant aléatoirement au préalable (indication: on pourra utiliser les fonctions `shuffle` de la librairie `random` et `train_test_split` de la librairie `sklearn.model_selection`)
    

In [504]:
# Sonar dataset #
sonar_data_fram = pd.read_table('Data/Sonar/sonar.all-data', sep = ',', header = None)
sonar_data_fram

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


- Fonction qui produit les listes de features et la liste de label
- Fonction qui génère deux listes de données `x_train` (75%) and `x_test` (25%) en la mélangeant aléatoirement au préalable

In [505]:
def produce_collection(dataFrame, numL, numC, targetC, classType=None):
    """
    Entree:
    dataFrame : le dataframe
    numL : nombre de lignes
    numC : nombre de colonnes
    targetC : colonne label
    classType : liste des classes (2 classes dans ce cas)
    
    Sortie:
    x : tableau des features
    y : tableau des labels 
    """
    x = np.zeros((numL, numC - 1))
    y = np.zeros(numL)
    
    feature_idx = 0  # indice pour les colonnes de x
    for i in range(numC):
        if i == targetC:
            continue  # on saute la colonne label
        x[:, feature_idx] = dataFrame[i]
        feature_idx += 1
    
    if classType == None:
        for i in range(numL):
            y[i] = dataFrame[targetC][i]
    else:
        for i in range(numL):
            if dataFrame[targetC][i] == classType[0]: y[i] = 1
            elif dataFrame[targetC][i] == classType[1]: y[i] = 0
            else: 
                print("Erreur: classType doit contenir 2 classes")
                exit(0)
            
    return x, y

def generate_train_test_random(x, y):
    """
    Entree:
    x : tableau des features
    y : tableau des labels
    
    Sortie:
    x_train : tableau des features pour l'entrainement
    x_test : tableau des features pour le test
    y_train : tableau des labels pour l'entrainement
    y_test : tableau des labels pour le test
    """ 
    data = list(zip(x, y))
    rd.shuffle(data)
    new_X = np.array([i for i, j in data])
    new_Y = np.array([j for i, j in data])
    x_train, x_test, y_train, y_test = train_test_split(new_X, new_Y, test_size=0.25)
    return x_train, x_test, y_train, y_test

<font color='red'><b>Question 4:</b></font> Appliquer ces modèles sur cette base (on pourra prendre $MaxEp\approx1000$ et le pas d'apprentissage $\eta\approx0.01$) et en choisissant les bases Train et Test de façon aléatoire 


In [724]:
def calculate_mean_error(collection, targetC, classType, num_iterations, MaxEp, eta, err_iter=False):
    x_collection, y_collection = produce_collection(collection, collection.shape[0], collection.shape[1], targetC, classType)
    x_train, x_test, y_train, y_test = generate_train_test_random(x_collection, y_collection)
    out = ""
    ada = np.zeros(num_iterations)
    rel = np.zeros(num_iterations)
    for _ in range(num_iterations):
        out += "iter " + str(_+1)  + ":\n"
        out += "Model Adaline: "
        w_adaline = SGD(x_train, y_train, MaxEp, eta, h, loss_adaline, adaline_grad, False)
        error_rate = []
        for x_i, y_i in zip(x_test, y_test):
            error_rate.append(loss_adaline(h(w_adaline, x_i), y_i))
        out += str(np.mean(error_rate)) + "\n"
        ada[_] = np.mean(error_rate)
        
        out += "Model Logistic Regression: "
        w_logistic = SGD(x_train, y_train, MaxEp, eta, prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, False)
        error_rate = []
        for x_i, y_i in zip(x_test, y_test):
            error_rate.append(loss_regression_logistique(prediction_regression_logistique(w_logistic, x_i), y_i))
        out += str(np.mean(error_rate)) + "\n"
        rel[_] = np.mean(error_rate)
    if err_iter:
        print(out)
    
    print(np.mean(ada), np.mean(rel))

- Reporter l'erreur moyenne de ces modèles obtenues sur les exemples de donnés de "test"?

In [725]:
class_type_sonar = ['M', 'R']
MaxEp = 1000
calculate_mean_error(sonar_data_fram, sonar_data_fram.shape[1] - 1, class_type_sonar, 1, MaxEp, eta_001, True)

iter 1:
Model Adaline: 0.18112945411483233
Model Logistic Regression: 0.6349223733147699

0.18112945411483233 0.6349223733147699


- Refaire l'opération 3 fois avec trois randomisations différentes. 

In [726]:
calculate_mean_error(sonar_data_fram, sonar_data_fram.shape[1] - 1, class_type_sonar, 3, MaxEp, eta_001, True)

iter 1:
Model Adaline: 0.14834995501534148
Model Logistic Regression: 0.6127587041779408
iter 2:
Model Adaline: 0.15428029991102285
Model Logistic Regression: 0.6318251859712103
iter 3:
Model Adaline: 0.12243317036184341
Model Logistic Regression: 0.6112738214479116

0.14168780842940257 0.6186192371990209


  | Collection | Adaline     | Régression Logistique |
  |------------|-------------|-----------------------|
  |   SONAR (réplica 1)   |            |           |
  |   SONAR (réplica 2)   |           |         |
  |   SONAR (réplica 3)   |           |           |


## Partie 3: normalisation

Nous allons étudier l'impact de la nomralisation sur les prédictions. Pour cela nous considérons deux stratégies de normalisation communément utilisées dans la littérature:
* Stratégie <i>max</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la valeur maximale de cette caractéristiques
* Stratégie <i>norme</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la norme de ce vecteur.

Nous considérons ces trois autres collections de la base UCI:

        * https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29
        * https://archive.ics.uci.edu/ml/datasets/spambase
        * https://archive.ics.uci.edu/ml/datasets/ionosphere

        

In [706]:
# Importer dataset #
breast_cancer_df = pd.read_table('Data/BreastCancer/wdbc.data', header=None, sep=',')
ionosphere_df = pd.read_csv('Data/Ionosphere/ionosphere.data', header=None, sep=',')
spambase_df = pd.read_csv('Data/Spambase/spambase.data', header=None, sep=',')

# breast_cancer_df
# ionosphere_df
# spambase_df


<font color='red'><b>Question 5:</b></font> Ecrire une fonction qui prend en entrée la collection des données et qui retourne la collections normalisée suivant les stratégies <i>max</i> et <i>norme</i>. 

- Choisir tous les colonnes contenant des données numériques et les convertir en type `float` pour é viter les problème de division.
-  <i>Max</i>: Diviser par la valeur maximale en valeur absolu
-  <i>Norme</i>: Norme L2 

In [707]:
def normalizer_collection(collection, strategies='max'):
    normalized_collection = collection.copy()
    
    numeric_cols = normalized_collection.select_dtypes(include=[np.number]).columns
    
    numeric_df = normalized_collection[numeric_cols].astype(float).copy()
    
    if strategies == 'max':
        max_value = np.max(np.abs(numeric_df), axis=0)
        max_value[max_value == 0] = 1 
        numeric_df = numeric_df / max_value
    
    elif strategies == 'norme': 
        l2_norm = np.linalg.norm(numeric_df, ord=2, axis=0)
        l2_norm[l2_norm == 0] = 1  
        numeric_df = numeric_df / l2_norm
        
    else:
        raise ValueError("Invalid normalization strategy")
    
    normalized_collection[numeric_cols] = numeric_df
    return normalized_collection


<font color='red'><b>Question 6:</b></font> Compléter les tableaux comparatifs suivants en repertant les erreurs moyennes sur 20 lancements des modèles de l'Adaline et de la Régression Logistique et pour les trois cas:




In [722]:
num_iterations = 20
breast_cancer_df
# calculate_mean_error(breast_cancer_df, 1, ['M', 'B'], num_iterations, MaxEp, eta_001)
# calculate_mean_error(ionosphere_df, ionosphere_df.shape[1] - 1, ['g', 'b'], num_iterations, MaxEp, eta_001)
# calculate_mean_error(sonar_data_fram, sonar_data_fram.shape[1] - 1, class_type_sonar, num_iterations, MaxEp, eta_001)
# calculate_mean_error(spambase_df, spambase_df.shape[1] - 1, None, num_iterations, MaxEp, eta_001)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400



 '*' Les vecteurs ne sont pas normalisés
     
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |


In [ ]:
# Normalisation avec strategie norme #
breast_cancer_df_norme = normalizer_collection(breast_cancer_df, 'norme')
ionosphere_df_norme = normalizer_collection(ionosphere_df, 'norme')
sonar_data_fram_norme = normalizer_collection(sonar_data_fram, 'norme')
spambase_df_norme = normalizer_collection(spambase_df, 'norme')

print("Breast Cancer:")
calculate_mean_error(breast_cancer_df_norme, 1, ['M', 'B'], num_iterations, MaxEp, eta_001)
print("Ionosphere:")
calculate_mean_error(ionosphere_df_norme, ionosphere_df_norme.shape[1] - 1, ['g', 'b'], num_iterations, 1000, eta_001)
print("Sonar:")
calculate_mean_error(sonar_data_fram_norme, sonar_data_fram_norme.shape[1] - 1, class_type_sonar, num_iterations, 1000, eta_001)
print("Spambase:")
calculate_mean_error(spambase_df_norme, spambase_df_norme.shape[1] - 1, None, num_iterations, 1000, eta_001)



Breast Cancer:
0.200095044600436 0.6897413022564496
Ionosphere:
0.29565636171942816 0.6781567460900401
Sonar:
0.2345651841815568 0.7017322227684746
Spambase:
0.00019086982303134217 0.6913760446149266


 
 $^n$ Normalisation suivant la stratégie <i>norme</i>
     
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |

  


In [717]:
# Normalisation avec strategie max #
breast_cancer_df_max = normalizer_collection(breast_cancer_df)
spambase_df_max = normalizer_collection(spambase_df)
ionosphere_df_max = normalizer_collection(ionosphere_df)

print("Breast Cancer:")
calculate_mean_error(breast_cancer_df_max, 1, ['M', 'B'], num_iterations, MaxEp, eta_001)
print("Ionosphere:")
calculate_mean_error(ionosphere_df_max, ionosphere_df_max.shape[1] - 1, ['g', 'b'], num_iterations, MaxEp, eta_001)
print("Sonar:")
calculate_mean_error(sonar_data_fram, sonar_data_fram.shape[1] - 1, class_type_sonar, num_iterations, MaxEp, eta_001)
print("Spambase:")
calculate_mean_error(spambase_df_max, spambase_df_max.shape[1] - 1, None, num_iterations, MaxEp, eta_001)


Breast Cancer:
0.08652205518087261 0.5373295590226499
Ionosphere:
0.19857414236777765 0.6327969576549477
Sonar:
0.20154379597658417 0.6742839942520126
Spambase:
0.2173167331479255 0.6807205917936906


 $^m$ Normalisation suivant la stratégie <i>max</i>
    
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |
